In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

2022-07-29 17:16:49.201793: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-29 17:16:49.201824: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
dftrain = pd.read_csv('Downloads/train.csv')
arr1 = np.zeros((42000, 785))
arr3 = np.zeros((42000, 784))
Ytrain = np.zeros((42000, 10))
Xtrain = np.zeros((42000, 28, 28, 1))
arr1 = np.array(dftrain)
arr2 = arr1[:, 1:]
Xtrain = arr2.reshape((42000, 28, 28, 1))
dftest = pd.read_csv('Downloads/test.csv')
arr3 = np.array(dftest)
Xtest = arr3.reshape((28000, 28, 28, 1))
for i in range(42000):
    Ytrain[i, arr1[i, 0]] = 1

In [12]:
def DigitModel():
    model = tf.keras.Sequential([
        tf.keras.Input(shape = (28, 28, 1)),
        tf.keras.layers.ZeroPadding2D(padding = 3),
        tf.keras.layers.Conv2D(32, 3, strides = 1),
        tf.keras.layers.Conv2D(64, 3, strides = 1),
        tf.keras.layers.BatchNormalization(axis = 3),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.ReLU(),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation = 'softmax')
    ])
    return model

In [13]:
DigitRecogModel = DigitModel()
DigitRecogModel.compile(optimizer = 'adam', loss = tf.keras.losses.CategoricalCrossentropy() , metrics = ['accuracy'])

In [14]:
DigitRecogModel.fit(Xtrain, Ytrain, epochs=13, batch_size = 64)

Epoch 1/13
657/657 [==============================] - 64s 97ms/step - loss: 0.3304 - accuracy: 0.9303
Epoch 2/13
657/657 [==============================] - 64s 97ms/step - loss: 0.1183 - accuracy: 0.9703
Epoch 3/13
657/657 [==============================] - 68s 103ms/step - loss: 0.0748 - accuracy: 0.9789
Epoch 4/13
657/657 [==============================] - 70s 106ms/step - loss: 0.0533 - accuracy: 0.9829
Epoch 5/13
657/657 [==============================] - 71s 108ms/step - loss: 0.0430 - accuracy: 0.9864
Epoch 6/13
657/657 [==============================] - 69s 105ms/step - loss: 0.0367 - accuracy: 0.9881
Epoch 7/13
657/657 [==============================] - 69s 105ms/step - loss: 0.0339 - accuracy: 0.9886
Epoch 8/13
657/657 [==============================] - 69s 106ms/step - loss: 0.0303 - accuracy: 0.9902
Epoch 9/13
657/657 [==============================] - 70s 106ms/step - loss: 0.0276 - accuracy: 0.9910
Epoch 10/13
657/657 [==============================] - 71s 108ms/step - los

In [15]:
pred_raw = DigitRecogModel.predict(Xtest, batch_size = 64)

438/438 [==============================] - 9s 20ms/step


In [16]:
pred_digit = np.zeros((28000,))
index = np.zeros((28000,))
pred_raw = (pred_raw>0.5)*1
# maxval = tf.math.reduce_max(pred_raw, axis = 1)
for i in range(28000):
    for j in range(10):
        if pred_raw[i,j] == 1:
            pred_digit[i] = j
    index[i] = i+1
pred_digit = pred_digit.astype('int64')
index = index.astype('int64')
print(pred_raw)
print(pred_digit)

[[0 0 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 1 ... 0 0 0]]
[2 0 9 ... 3 9 2]


In [17]:
data = {
    "ImageID" : index,
    "Label" : pred_digit
}
result = pd.DataFrame(data)
result.to_csv('Downloads/KaggleDigitRecognizer.csv', index = False)
print(result)

       ImageID  Label
0            1      2
1            2      0
2            3      9
3            4      9
4            5      3
...        ...    ...
27995    27996      9
27996    27997      7
27997    27998      3
27998    27999      9
27999    28000      2

[28000 rows x 2 columns]
